In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras.models import Sequential

model = Sequential()

In [4]:
log_mini= pd.read_csv("log_mini_finished.csv")
print(log_mini.columns)

FileNotFoundError: File b'log_mini_finished.csv' does not exist

In [5]:
songs= log_mini.drop(["session_id",'y1', 'y2'], axis=1)
print(songs.columns)
songs = np.array(songs)

Index(['duration', 'release_year', 'us_popularity_estimate', 'acousticness',
       'beat_strength', 'bounciness', 'danceability', 'dyn_range_mean',
       'energy', 'flatness', 'instrumentalness', 'liveness', 'loudness',
       'mechanism', 'organism', 'speechiness', 'tempo', 'time_signature',
       'valence', 'acoustic_vector_0', 'acoustic_vector_1',
       'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4',
       'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7', 'key_0',
       'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8',
       'key_9', 'key_10', 'mode_major'],
      dtype='object')


In [6]:
n_steps= 20
n_inputs=39
n_neurons=100
n_outputs=1
#batch_size = 128
n_iterations=1500
batch_size=20
learning_rate=0.001

In [7]:
X= tf.placeholder(tf.float32, [None,n_steps,n_inputs])
y= tf.placeholder(tf.float32, [None,n_outputs])

In [8]:
cell=tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu), output_size=n_outputs)

In [9]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [10]:
loss=tf.reduce_mean(tf.square(outputs-y))
optimizer= tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(loss)

In [11]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        train_y = np.array(log_mini['y2'])
        batch_x = songs[:batch_size*20,:].reshape((batch_size, 20, 39)) #should be (batch_size, timesteps, num_input)
        batch_y = train_y[:batch_size].reshape((batch_size, 1))
        #X_batch, y_batch= [songs.ix[iteration],2 ] # fetch the next training batch
        sess.run(training_op,feed_dict={X:batch_x, y:batch_y})
        if iteration%100== 0:
            mse= loss.eval(feed_dict={X:batch_x, y:batch_y})
            print(iteration, "\t MSE: " , mse)

NameError: name 'X_batch' is not defined